In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import requests
import numpy as np

In [2]:
def get_app_category(app_id):
    base_url = 'http://localhost:3000/api/apps/'
    response = requests.get(base_url+app_id)
    if response.status_code == 200:
        gid = response.json()["genreId"].upper()
        return gid
    else:
        print(app_id, " category cannot be found")
        return "NOT_FOUND"

In [14]:
def update_sheets(sheetname):
    df = pd.read_excel('RECORD_AUDIO.xlsx', sheet_name=sheetname)
    data = []
    for index, row in df.iterrows():
        data.append([row["Count"], row["Sentences"], row["Manually Marked"], row["uses-permission"], row["permission"]])
    updated_data = []
    app_order = 1
    description_tag = None
    for row in data:
        row[1] = str(row[1])
        if row[1].startswith("##"):
            if app_order != 1:
                description_row = [np.nan, "Description Tag", description_tag, np.nan, np.nan]
                updated_data.append(description_row)
            row[0] = "##{}".format(app_order)
            app_order += 1
            app_id = row[1].split("##")[1].strip()
            category = get_app_category(app_id)
            app_category = "CATEGORY/" + category
            category_row = [np.nan, app_category , np.nan, np.nan, np.nan]
            description_tag = row[2]
            row[2] = np.nan
            updated_data.append(row)
            updated_data.append(category_row)
        else:
            updated_data.append(row)
    return updated_data

In [18]:
from collections import defaultdict
def category_counts(sheetname):
    df = pd.read_excel('RECORD_AUDIO.xlsx', sheet_name=sheetname)
    data = []
    for index, row in df.iterrows():
        data.append([row["Count"], row["Sentences"], row["Manually Marked"], row["uses-permission"], row["permission"]])
    categories = defaultdict(int)
    for row in data:
        row[1] = str(row[1])
        if row[1].startswith("##"):
            app_id = row[1].split("##")[1].strip()
            category = get_app_category(app_id)
            categories[category] += 1
    return categories

In [30]:
def application_list(sheetname, outfile):
    df = pd.read_excel('RECORD_AUDIO.xlsx', sheet_name=sheetname)
    data = set()
   
    for index, row in df.iterrows():
        if row["Sentences"].startswith("##"):
            app_id = row["Sentences"].split("##")[1]
            data.add(app_id)
    return data


In [4]:
def write_to_excel(updated_data, output_file):
    def background_apply(value):
        if value.startswith("##") or value.startswith("Description Tag"):
            font_weight = 'bold'
        else:
            font_weight = 'normal'
        return 'font-weight: %s' % font_weight
    
    updated_df = pd.DataFrame(updated_data, columns = ["Count", "Sentences", "Manually Marked", "uses-permission", "permission"])
    styled = updated_df.style.applymap(background_apply, subset=['Sentences'])
    styled.to_excel(output_file, engine='openpyxl')

In [5]:
def run(sheet_name):
    updated_data = update_sheets(sheet_name)
    write_to_excel(updated_data, "{}.xlsx".format(sheet_name))

In [19]:
categories = category_counts("SEVIL")

com.duapps.recorder  category cannot be found


In [31]:
application_list("SEVIL", "record_audio_tagged_list.txt")

{'advanced3nd.ofamerican.english',
 'ae.uae.ott.hiu',
 'air.com.nhncorp.jrapp',
 'air.com.rosettastone.mobile.CoursePlayer',
 'alphastudio.adrama',
 'au.gov.wa.pta.transperth',
 'beauty.camera.photo.editor',
 'bts.dancing.line.tap.dash.music.zigzag',
 'call.free.international.phone.call',
 'chat.rocket.android',
 'city.russ.alltrackerfamily',
 'co.adultfnder.android',
 'co.clover.clover',
 'co.dilmil.android',
 'co.happybits.marcopolo',
 'co.triller.droid',
 'co.yellw.yellowapp',
 'co.zodiacmatch.freedating',
 'codematics.universal.tv.remote.control',
 'com.DreamonStudios.BouncyBasketball',
 'com.ElfizMedia.DubStepMusicMakerRM',
 'com.KABloomInteractive.EV3',
 'com.MixerStudio.SuperHearingSecretVoicesRecorderFREE',
 'com.Slack',
 'com.SouthernPacificOceanFisher.VoiceToText_memo',
 'com.a4tune',
 'com.ac.kannada.textbyvoice.speechtotext.voice.typingkeyboard',
 'com.airbnb.android',
 'com.alexvas.dvr',
 'com.alibaba.aliexpresshd',
 'com.alibaba.intl.android.apps.poseidon',
 'com.amazon.w

In [22]:
import operator
sorted_categories = sorted(categories.items(), key=operator.itemgetter(1), reverse=True)

total = 0
for line in sorted_categories:
    total += line[1]
    print(line[0], line[1])
print("Total : ", total)

MUSIC_AND_AUDIO 77
COMMUNICATION 68
EDUCATION 57
VIDEO_PLAYERS 51
PHOTOGRAPHY 43
TOOLS 41
SOCIAL 41
ENTERTAINMENT 36
PRODUCTIVITY 28
DATING 25
BUSINESS 18
TRAVEL_AND_LOCAL 16
MAPS_AND_NAVIGATION 15
LIFESTYLE 14
GAME_CASUAL 14
GAME_ACTION 11
NEWS_AND_MAGAZINES 10
HOUSE_AND_HOME 9
PERSONALIZATION 8
SHOPPING 8
GAME_ROLE_PLAYING 7
FINANCE 6
GAME_STRATEGY 6
GAME_TRIVIA 6
HEALTH_AND_FITNESS 5
PARENTING 5
BOOKS_AND_REFERENCE 5
GAME_MUSIC 4
SPORTS 4
GAME_EDUCATIONAL 4
MEDICAL 4
LIBRARIES_AND_DEMO 3
GAME_CARD 3
FOOD_AND_DRINK 2
GAME_SIMULATION 2
GAME_SPORTS 2
GAME_ADVENTURE 2
EVENTS 2
ART_AND_DESIGN 2
AUTO_AND_VEHICLES 1
GAME_RACING 1
GAME_PUZZLE 1
NOT_FOUND 1
BEAUTY 1
COMICS 1
GAME_ARCADE 1
Total :  671


In [26]:
new_categories = sorted_categories[len(sorted_categories)//2:]
for cat in new_categories:
    print('"{}",'.format(cat[0]), end =" ")

"GAME_TRIVIA", "HEALTH_AND_FITNESS", "PARENTING", "BOOKS_AND_REFERENCE", "GAME_MUSIC", "SPORTS", "GAME_EDUCATIONAL", "MEDICAL", "LIBRARIES_AND_DEMO", "GAME_CARD", "FOOD_AND_DRINK", "GAME_SIMULATION", "GAME_SPORTS", "GAME_ADVENTURE", "EVENTS", "ART_AND_DESIGN", "AUTO_AND_VEHICLES", "GAME_RACING", "GAME_PUZZLE", "NOT_FOUND", "BEAUTY", "COMICS", "GAME_ARCADE", 